In [2]:
import numpy as np


In [3]:
setup_time = np.array([
    [np.inf, 15.05, 12.37, 14.78, 15.23, np.inf],
    [13.70,  np.inf, 15.67, 13.12, 15.46, np.inf],
    [13.51, 13.91,  np.inf, 11.60, 11.86, np.inf],
    [16.49, 16.93, 13.55,  np.inf, 11.88, np.inf],
    [17.00, 16.56, 17.03, 17.28,  np.inf,np.inf],
    [10.58, 16.04, 13.82, 11.54, 14.88, np.inf] 
], dtype=float)

In [4]:

def tsp_nearest_neighbor(distance_matrix, start):
    n = len(distance_matrix)
    visited = [False] * n
    tour = [start]
    visited[start] = True
    total_distance = 0

    current_city = start
    for _ in range(n - 1):
        nearest_city = None
        nearest_distance = float("inf")
        for city in range(n):
            if not visited[city] and distance_matrix[current_city][city] < nearest_distance:
                nearest_city = city
                nearest_distance = distance_matrix[current_city][city]
        
        if nearest_city is None:  # safeguard
            break

        tour.append(nearest_city)
        visited[nearest_city] = True
        total_distance += nearest_distance
        current_city = nearest_city

    return tour, total_distance


tour, distance = tsp_nearest_neighbor(setup_time, start=5)
print("Nearest Neighbor Tour:", tour)
print("Total Distance:", distance)


Nearest Neighbor Tour: [5, 0, 2, 3, 4, 1]
Total Distance: 62.989999999999995


In [5]:
def greedy_tsp_asymmetric(distance_matrix):
    n = len(distance_matrix)
    edges = []

    # Step 1: collect valid directed edges
    for i in range(n):
        for j in range(n):
            if i != j and distance_matrix[i][j] != float("inf"):
                edges.append((distance_matrix[i][j], i, j))
    
    # Step 2: sort edges by weight
    edges.sort(key=lambda x: x[0])

    out_degree = [0] * n  # how many edges go out from each node
    in_degree = [0] * n   # how many edges go into each node
    tour_edges = []

    # Step 3: select edges
    for w, u, v in edges:
        if out_degree[u] < 1 and in_degree[v] < 1:
            # Avoid forming cycle before including all nodes
            if len(tour_edges) < n - 1 or (v == tour_edges[0][0] and u not in [x[0] for x in tour_edges]):
                tour_edges.append((u, v))
                out_degree[u] += 1
                in_degree[v] += 1
        if len(tour_edges) == n:
            break

    # Step 4: reconstruct tour from directed edges
    tour = [tour_edges[0][0]]
    current = tour_edges[0][0]
    while len(tour) < n:
        for u, v in tour_edges:
            if u == current:
                tour.append(v)
                current = v
                break
    # Step 5: compute distance
    total_distance = 0
    for i in range(len(tour)-1):
        total_distance += distance_matrix[tour[i]][tour[i+1]]

    return tour, total_distance


In [6]:
greedy_tsp_asymmetric(setup_time)

([5, 0, 2, 3, 4, 1], np.float64(62.989999999999995))

In [10]:
setup_atime = np.array([
    [np.inf, 15.05, 12.37, 14.78, 15.23],
    [13.70,  np.inf, 15.67, 13.12, 15.46],
    [13.51, 13.91,  np.inf, 11.60, 11.86 ],
    [16.49, 16.93, 13.55,  np.inf, 11.88],
    [17.00, 16.56, 17.03, 17.28,  np.inf],
    [10.58, 16.04, 13.82, 11.54, 14.88] 
], dtype=float)


In [14]:
from python_tsp.exact import solve_tsp_dynamic_programming

permutation, distance = solve_tsp_dynamic_programming(setup_time)
start_node = 5
if start_node in permutation:
    idx = permutation.index(start_node)
    permutation = permutation[idx:] + permutation[:idx]

# Recalculate distance for this rotated tour
new_distance = 0
for i in range(len(permutation)):
    u = permutation[i]
    v = permutation[(i + 1) % len(permutation)]  # wrap around to make it a cycle
    new_distance += setup_time[u, v]

print("Tour starting at node 5:", permutation)
print("Recalculated distance:", new_distance)



Tour starting at node 5: [5, 0, 1, 2, 3, 4]
Recalculated distance: inf


In [8]:
permutation

[0, 1, 2, 3, 4, 5]

In [9]:
distance

np.float64(inf)